In [1]:
import pandas as pd, ipywidgets as widgets, numpy as np, matplotlib.pyplot as plt
import io as io
from ipywidgets import interact, interactive, fixed, interact_manual
raw_uploader, eln_uploader = widgets.FileUpload(multiple = False), widgets.FileUpload(multiple = False)
display("Upload iQue3 Data (csv)")
display(raw_uploader)
display("Upload ELN w/ Antibodies Sheet (xlsx)")
display(eln_uploader)

'Upload iQue3 Data (csv)'

FileUpload(value={}, description='Upload')

'Upload ELN w/ Antibodies Sheet (xlsx)'

FileUpload(value={}, description='Upload')

In [3]:
iquename, elnname = list(raw_uploader.value.keys())[0], list(eln_uploader.value.keys())[0]
iquedata = pd.read_csv(io.BytesIO(raw_uploader.value[iquename]['content']))
eln = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']))
ab_sheet = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), sheet_name = "Antibody Plate Map")
iquedata.rename(columns = {'Plate' : 'iQue3 Plate','Well ID':'iQue3 384 Well ID'}, inplace = True)
ab_sheet.rename(columns = {'Plate' : 'Antibody Plate','Well ID':'Antibody Platemap 96 Well ID'}, inplace = True)
data_cleaned = iquedata[["iQue3 Plate", "iQue3 384 Well ID", "Well Type", "Median RL1-H of singlets", "Count of singlets"]]
data_cleaned

IndexError: list index out of range

In [3]:
#Return cleaned iQue3 data with only samples of the well type specified
def clean_for_well_type(well_type_string):
    return data_cleaned[data_cleaned["Well Type"] == well_type_string]

#Average function that takes a string indicating which well type to average (well_type_string)
#Also takes a string indicating what data from the inidicated well type to average (to_average_string)
#ie. "Median RL1H" or "Cell Count"(
#Separates n=1 and n=2 replicates from 384 well format, resets indeces, and averages
#Returns a series with the averaged MFI ratios, removing the replicate wells
def average(well_type_string, to_average_string):
    selected_wells = clean_for_well_type(well_type_string)
    values_to_average = selected_wells[to_average_string]
    return average_series(values_to_average)

#Average function that takes a series with n=1, n=2 replicates in sequential order within the series, and 
#Separates n=1 and n=2 replicates resets indeces, and calculates average
#Returns a series with the averaged MFI ratios, removing the replicate wells
def average_series(series_to_average):
    firsts = series_to_average.iloc[::2]
    seconds = series_to_average.iloc[1::2]
    firsts.reset_index(inplace = True, drop = True)
    seconds.reset_index(inplace = True, drop = True)
    averaged_series = firsts + seconds
    averaged_series = averaged_series/2
    return averaged_series


#Calculates the MFI ratio given a positive and negative series, 
#and appends the MFI ratio to the original data frame, excluding the negative population
#Calculates the average cell count for each duplicate, and appends average cell count to original data frame
def calculate_analysis_sheet(positive, negative):
    MFI_ratio = positive/negative
    original_positive = clean_for_well_type("Positive")
    original_positive = original_positive[::2]
    original_positive.reset_index(inplace = True, drop = True)
    original_positive["MFI Ratio"] = MFI_ratio
    MFIs_Antibody = pd.concat([original_positive, ab_sheet], axis = 1)
    MFIs_Antibody.index += 1
    #averagePositiveCounts = average("Positive", "Count of singlets")
    return MFIs_Antibody

In [4]:
#Create average MFI of positive population series and average MFI of negative population series
averagePositive = average("Positive", "Median RL1-H of singlets")
averageNegative = average("Negative", "Median RL1-H of singlets")

#Create new DF for MFI ratios(don't modify originals)
analyzed_data = calculate_analysis_sheet(averagePositive, averageNegative)
analyzed_data.to_excel("Flow Binding MFI Ratios.xlsx", sheet_name='Analyzed Data')
AllLeadsSorted = analyzed_data.sort_values(by="MFI Ratio", ascending = False)
analyzed_data
#AllLeadsSorted

,iQue3 Plate,iQue3 384 Well ID,Well Type,Median RL1-H of singlets,Count of singlets,MFI Ratio,Lot #,Protein Name,Antibody Platemap 96 Well ID,Protein Concentration (ug/uL),Date Purified,BPP,Antibody Plate,Original Plate ID/Barcode
1,Plate 2,A01,Positive,221357.0,1997.0,18.075564,57063.0,TB734B-101,A1,0.88,10.25.22,662.0,1.0,BPP662
2,Plate 2,C01,Positive,935017.5,2306.0,2.244292,57064.0,TB734C-101,B1,0.11,10.25.22,662.0,1.0,BPP662
3,Plate 2,E01,Positive,29070.0,2639.0,2.810419,57065.0,TB735A-101,C1,0.51,10.25.22,662.0,1.0,BPP662
4,Plate 2,G01,Positive,122462.0,1843.0,7.930800,57066.0,TB735A-102,D1,0.41,10.25.22,662.0,1.0,BPP662
5,Plate 2,I01,Positive,89338.0,2865.0,5.095377,57067.0,TB735A-103,E1,0.11,10.25.22,662.0,1.0,BPP662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D11,TB735A-083,-0.08,2022-10-22 00:00:00,616.0,3.0,BPP-616 R-IGGP-383-221017.
205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E11,TB735A-084,0.19,2022-10-22 00:00:00,616.0,3.0,BPP-616 R-IGGP-383-221017.
206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F11,TB735A-085,-0.06,2022-10-22 00:00:00,616.0,3.0,BPP-616 R-IGGP-383-221017.
207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G11,TB735A-086,-0.07,2022-10-22 00:00:00,616.0,3.0,BPP-616 R-IGGP-383-221017.


In [ ]:
# #Sort DF by highest MFI ratio (top hits)
# AllLeadsSorted = MFIRatio_Antibody_cleaned.sort_values(by="MFI Ratio", ascending = False)
# Top10 = MFIRatio_Antibody_cleaned.nlargest(10, "MFI Ratio")
# #Top10 = Top10.append(MFIRatio_Antibody_cleaned.loc[MFIRatio_Antibody_cleaned["DNA"] == "SC-175-017"])
# fig, ax = plt.subplots()
# ax.scatter(x = AllLeadsSorted["Protein Concentration (ug/uL)"], y=AllLeadsSorted["MFI Ratio"], alpha = .2, color = "blue")
# ax.scatter(x = Top10["Protein Concentration (ug/uL)"], y = Top10["MFI Ratio"], alpha = .8, color = "blue", label = "Top Hits")
# ax.set_title("CD40 Primary Screen")
# ax.set_xlabel("Concentration (mg/ml)")
# ax.set_ylabel("MFI Ratio \n Transfected/Untransfected")
# ax.legend()

# texts = []
# for row in Top10.itertuples(index = False):
#     ab, conc, MFI = row[4], row[5], row[3]
#     ax.annotate(row[4], xy = (conc, MFI), textcoords = "offset points", xytext = (8, -2), ha ='left')
#     #texts.append(plt.text(ab, conc, MFI))